* ### encode related

In [23]:
# dedicated for vgg16, may not be compatitable with other networks without kernel size equal to 3 and padding size equal to 1

import numpy as np

class EncoderClass():
    def fullzero(length : int) -> str:
        assert length <= 2 ** 34
        return '01{:0>34b}'.format(length - 1)
    

    def adddata(length : int, baseaddr : int, padding : bool, repeat : int) -> str:
        assert length <= 2 ** 8
        assert repeat <= 2 ** 8
        assert baseaddr < 2 ** 17

        return '10{:0>17b}{:0>8b}{:0>8b}{:0>1b}'.format(baseaddr, length - 1, repeat - 1, padding)

    


class SingleImageClass():
    
    def ImageEncode(self, image : np.array, baseaddr : int) -> list:
        IMAGE_CHANNEL_INDEX = 0
        IMAGE_WIDTH_INDEX = 1
        IMAGE_HEIGHT_INDEX = 2
        
        ImageInstructions_set = [list() for i in range(0, 9)]
        addroffset = baseaddr
        for ImagePerChannel in image:
            ImagePerChannelInstructions_set = self.ImagePerChannelEncode(ImagePerChannel, addroffset, 3, True)

            for i in range(0, 9):
                ImageInstructions_set[i].extend(ImagePerChannelInstructions_set[i])

            addroffset = addroffset + image.shape[IMAGE_WIDTH_INDEX] * image.shape[IMAGE_HEIGHT_INDEX]

        return ImageInstructions_set
    

    def ImagePerChannelEncode(self, image : np.array, baseaddr : int, KernelSize : int = 3, padding : bool = True) -> list:
        IMAGE_WIDTH_INDEX = 0
        IMAGE_HEIGHT_INDEX = 1

        assert baseaddr < 2 ** 17   # print("channel's base address overflow")
         
        assert padding == True      # print("padding unsupported")
        assert KernelSize == 3      # print("Kernel Size unsupported")
            

        instructions_set = [list() for i in range(0, 9)]

        ### block_x1y1
        instructions_set[0].append(EncoderClass.fullzero(image.shape[IMAGE_WIDTH_INDEX] + 1))
        instructions_set[0].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr, True, image.shape[IMAGE_HEIGHT_INDEX] - 2))
        addroffset = image.shape[IMAGE_WIDTH_INDEX] * (image.shape[IMAGE_HEIGHT_INDEX] - 2)
        # print(addroffset)
        instructions_set[0].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX] - 1, baseaddr + addroffset, False, 1))

        ### block_x2y1
        instructions_set[1].append(EncoderClass.fullzero(image.shape[IMAGE_WIDTH_INDEX]))
        instructions_set[1].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr, False, image.shape[IMAGE_HEIGHT_INDEX] - 1))

        ### block_x3y1
        instructions_set[2].append(EncoderClass.fullzero(image.shape[IMAGE_WIDTH_INDEX]))
        instructions_set[2].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr + 1, True, image.shape[IMAGE_HEIGHT_INDEX] - 1))

        ### block_x1y2
        instructions_set[3].append(1)
        instructions_set[3].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr, True, image.shape[IMAGE_HEIGHT_INDEX] - 1))
        addroffset = image.shape[IMAGE_WIDTH_INDEX] * (image.shape[IMAGE_HEIGHT_INDEX] - 1)
        instructions_set[3].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX] - 1, baseaddr + addroffset, False, 1))

        ### block_x2y2
        instructions_set[4].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr, False, image.shape[IMAGE_HEIGHT_INDEX]))

        ### block_x3y2
        instructions_set[5].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr + 1, True, image.shape[IMAGE_HEIGHT_INDEX]))

        ### block_x1y3
        instructions_set[6].append(1)
        instructions_set[6].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr + image.shape[IMAGE_WIDTH_INDEX], True, image.shape[IMAGE_HEIGHT_INDEX] - 1))
        instructions_set[6].append(EncoderClass.fullzero(image.shape[IMAGE_WIDTH_INDEX] - 1))

        ### block_x2y3
        instructions_set[7].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr + image.shape[IMAGE_WIDTH_INDEX], False, image.shape[IMAGE_HEIGHT_INDEX] - 1))
        instructions_set[7].append(EncoderClass.fullzero(image.shape[IMAGE_WIDTH_INDEX]))

        ### block_x3y3
        instructions_set[8].append(EncoderClass.adddata(image.shape[IMAGE_WIDTH_INDEX], baseaddr + image.shape[IMAGE_WIDTH_INDEX] + 1, True, image.shape[IMAGE_HEIGHT_INDEX] - 1))
        instructions_set[8].append(EncoderClass.fullzero(image.shape[IMAGE_WIDTH_INDEX]))

        # for pix_posx in range(0, image.len(0) + 2, 7):
        #     for pix_posy in range(0, image.len(1) + 2, 7):
        #         if pix_posx == 0 and pix_posy == 0:                                 # left top

        #         elif pix_posx == image.len(0) + 1 and pix_posy == 0:                # right top

        #         elif pix_posx == 0 and pix_posy == image.len(1) + 1:                # left bottom
                
        #         elif pix_posx == image.len(0) + 1 and pix_posy == image.len(1) + 1: # right bottom

        #         else:

        return instructions_set
    

#####  example code  ######

# SingleImage_o = SingleImageClass()
# input_channel = np.zeros((224, 224))

# output_intruction = SingleImage_o.ImagePerChannelEncode(input_channel, 0)


* ### model related

In [31]:
class VggArgument:
    size = 0
    baseaddr = 0

def CombineChannelsToLayer(size, channelamount, baseaddr):
    vgg_arch_arguments_channel = []
    for channel in range(0, channelamount):
        VggArgument_o = VggArgument()
        VggArgument_o.size = size
        VggArgument_o.baseaddr = baseaddr

        vgg_arch_arguments_channel.append(VggArgument_o)
    return vgg_arch_arguments_channel

vgg_arch_arguments = []

## conv1
vgg_arch_arguments_layer = []

vgg_arch_arguments_layer.append(CombineChannelsToLayer(224, 3, 0)) ### conv1_1
vgg_arch_arguments_layer.append(CombineChannelsToLayer(224, 64, 0)) ### conv1_2

vgg_arch_arguments.append(vgg_arch_arguments_layer)


## conv2
vgg_arch_arguments_layer = []

vgg_arch_arguments_layer.append(CombineChannelsToLayer(112, 64, 0)) ### conv2_1
vgg_arch_arguments_layer.append(CombineChannelsToLayer(112, 128, 0)) ### conv2_2

vgg_arch_arguments.append(vgg_arch_arguments_layer)


## conv3
vgg_arch_arguments_layer = []

vgg_arch_arguments_layer.append(CombineChannelsToLayer(56, 128, 0)) ### conv3_1
vgg_arch_arguments_layer.append(CombineChannelsToLayer(56, 256, 0)) ### conv3_2
vgg_arch_arguments_layer.append(CombineChannelsToLayer(56, 256, 0)) ### conv3_3

vgg_arch_arguments.append(vgg_arch_arguments_layer)


## conv4
vgg_arch_arguments_layer = []
vgg_arch_arguments_layer.append(CombineChannelsToLayer(28, 256, 0)) ### conv4_1
vgg_arch_arguments_layer.append(CombineChannelsToLayer(28, 512, 0)) ### conv4_2
vgg_arch_arguments_layer.append(CombineChannelsToLayer(28, 512, 0)) ### conv4_3

vgg_arch_arguments.append(vgg_arch_arguments_layer)


## conv5
vgg_arch_arguments_layer = []

vgg_arch_arguments_layer.append(CombineChannelsToLayer(14, 512, 0)) ### conv5_1
vgg_arch_arguments_layer.append(CombineChannelsToLayer(14, 512, 0)) ### conv5_2
vgg_arch_arguments_layer.append(CombineChannelsToLayer(14, 512, 0)) ### conv5_3

vgg_arch_arguments.append(vgg_arch_arguments_layer)



* ### encode model input

In [34]:
SingleImage_o = SingleImageClass()
total_instruction = [list() for i in range(0, 9)]

for cur_layer in vgg_arch_arguments:
    for cur_conv in cur_layer:
        for cur_channel in cur_conv:
            input_channel = np.zeros((cur_channel.size, cur_channel.size))
            output_instruction = SingleImage_o.ImagePerChannelEncode(input_channel, cur_channel.baseaddr)
            for i in range(0, 9):
                total_instruction[i].extend(output_instruction[i])


In [36]:
for i in range(0, 9):
    print(len(total_instruction[i]))

11145
7430
7430
11145
3715
3715
11145
7430
7430
